Application that reads a table, displays it, and loads an image upon clicking a cell in the "name" column, follow these steps. It used single images output from the ```GranularityDataGen().granCalc_imageGen()``` function

In [ ]:
!pip install --upgrade plotly
!pip install jupyter-dash
!pip install dash-bootstrap-components

In [ ]:
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output
from dash import State
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import os
import re
import glob
import random
import plotnine
from sklearn import preprocessing
from tqdm import tqdm
import plotly.express as px
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')
import pdb
# mount data
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
dataPath = '/content/drive/MyDrive/Data/granularity/imageSequence/data'
IMAGE_FOLDER = '/content/drive/MyDrive/Data/granularity/imageSequence/images'
df = pd.read_csv(os.path.join(dataPath,'imageseq_data.csv'))
float_cols = df.select_dtypes(include=['float']).columns
df[float_cols] = df[float_cols].apply(lambda x: np.round(x, 3))
#df['name'] = df['name'].apply(lambda x: f"{x}.png")
first_image_name = df.iloc[0]['name']
first_image_path = f"{IMAGE_FOLDER}/{first_image_name}"
savedContainer = {'name':[], 'ratio':[], 'intensity':[], 'sd':[], 'maskArea':[], 'label':[]}
image_names = [re.sub("_.*","",name) for name in set(df.name.tolist())]
#df.loc[df.name.str.contains(image_names[0]),:]

In [ ]:
def addimagetoDict(dictContainer,curr_container):
  dictContainer['name'].append(curr_container['name'])
  dictContainer['ratio'].append(curr_container['ratio'])
  dictContainer['intensity'].append(curr_container['intensity'])
  dictContainer['sd'].append(curr_container['sd'])
  dictContainer['maskArea'].append(curr_container['maskArea'])
  dictContainer['label'].append(curr_container['label'])


In [ ]:
# Initialize the Jupyter Dash app
app = JupyterDash(__name__, suppress_callback_exceptions=True)

# Define the layout of the app
app.layout = html.Div([
       html.Div([
        dcc.Dropdown(image_names,id="dropdown",),
        html.Div(id='table-id'),
        html.Br(),
        html.Button("Save Selected Row", id="save-btn", n_clicks=0),
        html.Div(id="saved-row"),
        html.Br(),
        html.Div(id = 'output-save-csv', children=[
          dcc.Input(id="input2", type="text", placeholder="Insert file name", debounce=True )],style={'width': '49%', 'display': 'inline-block'}),
        html.Div([
          html.Button("Save data", id="save-csv", n_clicks=0)],style={'width': '29%', 'display': 'inline-block'}),
        ], style={'width': '49%', 'display': 'inline-block'}),
      dcc.Store(id='table-all',data=None),
      dcc.Store(id='dict-container',data=None),
       html.Div([
    html.Div(id='img-container')
  ], style={'width': '49%', 'display': 'inline-block'})
])


@app.callback(
  [Output('table-id', 'children'),
   Output('table-all', 'data')],
  Input('dropdown', 'value'),
  prevent_initial_call=True,
)
def select_image_name(image_name_sel):
  df_new = df.loc[df.name.str.contains(image_name_sel),:].reset_index()
  return [dash_table.DataTable(
          id='table',
          columns=[{"name": i, "id": i,}
                  for i in df.columns if i in ['name', 'ratio', 'label']],
          data=df_new.to_dict('records'),
          row_selectable="single",
          editable = True,
          page_size=10,
          cell_selectable=True)],df_new.to_dict('records')

@app.callback(
  Output('table', 'data',allow_duplicate=True),
  [Input('table', 'data'),
  Input('table', 'active_cell')],
  prevent_initial_call=True,
)
def update_data(data,active_cell):
  # Handler for data update; includes save logic as needed
  if active_cell['column_id'] == 'label':
      return data.to_dict('records')
  else:
    raise PreventUpdate

@app.callback(
  Output('table', 'data'),
  [Input('table', 'page_current'),
  Input('table-all', 'data'),
   State('table', 'page_size')],
  prevent_initial_call=True,
)
def page_update(page_current,data,page_size):
  # Handler for data update; includes save logic as needed
  if page_current is None:
    raise PreventUpdate
  else:
    df_all = pd.DataFrame(data)
    return df_all.iloc[page_current*page_size:(page_current+1)*page_size].to_dict('records')

@app.callback(
  [Output("saved-row", "children"),
   Output("dict-container", "data")],
  Input("save-btn", "n_clicks"),
  Input("table", "selected_rows"),
  State("table", "data"),
  prevent_initial_call=True,
)
def save_row(n_clicks, selected_rows, rows):
  if selected_rows:
      selected_row_data = rows[selected_rows[0]]
      addimagetoDict(dictContainer = savedContainer,curr_container = selected_row_data)
      # Save logic for the selected row
      return [f"Number of rows: {len(savedContainer['name'])}",savedContainer]
  return ["No row selected to save.",None]

@app.callback(
  Output("output-save-csv", "children"),
  Input("input2", "value"),
  Input("save-csv", "n_clicks"),
  State("dict-container", "data"),
  prevent_initial_call=True,
)
def save_data(image_name, click_save, container):
  if image_name is None or container is None:
    return PreventUpdate
  else:
    df = pd.DataFrame(container)
    df.to_csv(os.path.join(dataPath,image_name + '.csv'))
    return "data is saved"


@app.callback(
  Output('img-container', 'children'),
  Input('table', 'active_cell'),
  Input('table', 'data'),
  prevent_initial_call=True,
)
def display_image(active_cell, rows):
  if active_cell['column_id'] == 'name':
    img_name = rows[active_cell['row']]['name']  # No need to append ".png"
    img_path = f"{IMAGE_FOLDER}/{img_name}"
    im_pil = Image.open(img_path)
    img = px.imshow(im_pil,binary_string=True, binary_backend="png", width=500, height=500,binary_compression_level=9).update_xaxes(showticklabels=False).update_yaxes(showticklabels = False)
    return [f'{active_cell}']
  else:
    raise PreventUpdate


# @app.callback(
#   Output('img-container', 'children'),
#   Input('table', 'active_cell'),
#   Input('table', 'data'),
#   prevent_initial_call=True,
# )
# def display_image(active_cell, rows):
#   if active_cell['column_id'] == 'name':
#     img_name = rows[active_cell['row']]['name']  # No need to append ".png"
#     img_path = f"{IMAGE_FOLDER}/{img_name}"
#     im_pil = Image.open(img_path)
#     img = px.imshow(im_pil,binary_string=True, binary_backend="png", width=500, height=500,binary_compression_level=9).update_xaxes(showticklabels=False).update_yaxes(showticklabels = False)
#     return [dcc.Graph(id="graph_ch1",figure=img)]
#   else:
#     return PreventUpdate

# Run the server
app.run_server()


To display the first image upon launching the app, you need to set the `src` of `html.Img` based on the first row's 'name'. Update the initialization of the image container in your app layout to include the first image. Adjust the path as needed for your setup.

In [ ]:
# Initialize the Jupyter Dash app
app = JupyterDash(__name__)

# Define the layout of the app
app.layout = html.Div([
       html.Div([
        dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i, "editable": True if i == "label" else False}
                 for i in df.columns if i in ['name', 'ratio', 'label']],
        data=df.to_dict('records'),
        row_selectable="single",
        page_size=10,
        cell_selectable=True),
        html.Br(),
        html.Button("Save Selected Row", id="save-btn", n_clicks=0),
        html.Div(id="saved-row")
        ], style={'width': '49%', 'display': 'inline-block'}),
      html.Div([
    html.Div(id='img-container')
  ], style={'width': '49%', 'display': 'inline-block'})
])

@app.callback(
  Output('table', 'data'),
  [Input('table', 'data'),
  Input('table', 'data_previous'),
  State('table', 'data_timestamp')],
  prevent_initial_call=True,
)
def update_data(data,data_previous, timestamp):
  # Handler for data update; includes save logic as needed
  if data_previous is None:
      raise PreventUpdate
  # Assuming DataFrame 'df' updates or saves happen here
  else:
    # df_cur = pd.DataFrame(data)
    # df_cur.to_csv(os.path.join(dataPath, 'imageseq_data.csv'), index=False)
    # #     data_previous.to_csv(os.path.join(dataPath, 'imageseq_data.csv'), index=False)
    return df_cur.to_dict('records')

@app.callback(
  Output("saved-row", "children"),
  Input("save-btn", "n_clicks"),
  State("table", "selected_rows"),
  State("table", "data"),
  prevent_initial_call=True,
)
def save_row(n_clicks, selected_rows, rows):
  if selected_rows:
      selected_row_data = rows[selected_rows[0]]
      # Save logic for the selected row
      return f"Saved row: {selected_row_data}"
  return "No row selected to save."


@app.callback(
  Output('img-container', 'children'),
  Input('table', 'active_cell'),
  Input('table', 'data'),
  prevent_initial_call=True,
)
def display_image(active_cell, rows):
  if active_cell['column_id'] == 'name':
    img_name = rows[active_cell['row']]['name']  # No need to append ".png"
    return f'{active_cell}'


# @app.callback(
#   Output('img-container', 'children'),
#   Input('table', 'active_cell'),
#   State('table', 'data'),
#   prevent_initial_call=True,
# )
# def display_image(active_cell, rows):
#   if active_cell['column_id'] == 'name':
#     img_name = rows[active_cell['row']]['name']  # No need to append ".png"
#     img_path = f"{IMAGE_FOLDER}/{img_name}"
#     im_pil = Image.open(img_path)
#     img = px.imshow(im_pil,binary_string=True, binary_backend="png", width=500, height=500,binary_compression_level=9).update_xaxes(showticklabels=False).update_yaxes(showticklabels = False)
#     return [dcc.Graph(id="graph_ch1",figure=img)]


# Run the server
app.run_server()
#   im_pil = Image.open(first_image_path)
#   img = px.imshow(im_pil,binary_string=True, binary_backend="png", width=500, height=500,binary_compression_level=9).update_xaxes(showticklabels=False).update_yaxes(showticklabels = False)
#   return dcc.Graph(id="graph_ch2",figure=img)b